In [1]:
%%capture
!pip install langchain==0.2.5 faiss-cpu==1.8.0 cohere==5.5.8 langchain-community==0.2.5 rank_bm25==0.2.2 sentence-transformers==3.0.1
!pip install llama-cpp-python==0.2.78  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

There’s a lot of research on how to best use language models for search. Three
broad categories of these models are dense retrieval, reranking, and RAG.

# Dense Retrieval Example


Dense retrieval systems rely on the concept of embeddings, the same concept
we’ve encountered in the previous chapters, and turn the search problem into
retrieving the nearest neighbors of the search query (after both the query and
the documents are converted into embeddings). Figure 8-1 shows how dense
retrieval takes a search query, consults its archive of texts, and outputs a set of
relevant results.

Let’s take a look at a dense retrieval example by using Cohere to search the Wikipedia
page for the film Interstellar. In this example, we will do the following:
1. Get the text we want to make searchable and apply some light processing to
chunk it into sentences.
2. Embed the sentences.
3. Build the search index.
4. Search and see the results.

## 1. Getting the text archive and chunking it

In [2]:

import cohere

# Paste your API key here. Remember to not share publicly
api_key = ''

# Create and retrieve a Cohere API key from os.cohere.ai
co = cohere.Client(api_key)

In [1]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""

# Split into a list of sentences
texts = text.split('.')

# Clean up to remove empty spaces and new lines
texts = [t.strip(' \n') for t in texts]

In [2]:
texts

['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
 'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
 'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
 'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
 'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
 'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',
 'Interstellar uses extensive practical a

## 2. Embedding the Text Chunks


Let’s now embed the texts. We’ll send them to the Cohere
API, and get back a vector for each text:

In [ ]:
import numpy as np

# Get the embeddings
response = co.embed(
  texts=texts,
  input_type="search_document",
).embeddings

embeds = np.array(response)
print(embeds.shape)

This outputs (15, 4096), which indicates that we have 15 vectors, each one of size
4,096.

## 3. Building The Search Index
Before we can search, we need to build a search index.
An index stores the embeddings and is optimized to quickly retrieve the nearest
neighbors even if we have a very large number of points:

In [ ]:
import faiss

dim = embeds.shape[1]
index = faiss.IndexFlatL2(dim)
index.add(np.float32(embeds))

## 4. Search the index
We can now search the dataset using any query we want. We simply
embed the query and present its embedding to the index, which will retrieve the most
similar sentence from the Wikipedia article.
Let’s define our search function:

In [ ]:
import pandas as pd

def search(query, number_of_results=3):

  # 1. Get the query's embedding
  query_embed = co.embed(texts=[query],
                input_type="search_query",).embeddings[0]

  # 2. Retrieve the nearest neighbors
  distances , similar_item_ids = index.search(np.float32([query_embed]), number_of_results)

  # 3. Format the results
  texts_np = np.array(texts) # Convert texts list to numpy for easier indexing
  results = pd.DataFrame(data={'texts': texts_np[similar_item_ids[0]],
                              'distance': distances[0]})

  # 4. Print and return the results
  print(f"Query:'{query}'\nNearest neighbors:")
  return results

In [ ]:
query = "how precise was the science"
results = search(query)
results

| texts | distance |
| --- | --- |
| It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics | 10757.379883 |
| Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar | 11566.131836 |
| Interstellar uses extensive practical and mini... | 11922.833008 |


The first result has the least distance, and so is the most similar to the query. Looking
at it, it answers the question perfectly. Notice that this wouldn’t have been possible if
we were only doing keyword search because the top result did not include the same
keywords in the query.

### Disadvantages of Dense Retrieval

#### 1. Irrelevant Results When the Answer Is Missing
- If the retrieved texts do not contain the answer, dense retrieval still returns results based on similarity scores.
- **Example**: A query like *"What is the mass of the moon?"* might return unrelated results about movies or cinematography.
- This happens because dense retrieval is based on embeddings and similarity, not direct keyword matching.

#### 2. Handling Long Texts (Chunking Challenges)
- **Transformer models have a limited context size**, restricting the number of tokens they can process at once.
- Long documents need to be split into smaller chunks, but how this is done affects retrieval quality.

##### Chunking Strategies:
- **One Vector per Document:**
  - Embedding only a representative part (e.g., title or introduction) leaves out a lot of information.
  - Averaging embeddings from multiple chunks compresses information, reducing accuracy.
- **Multiple Vectors per Document:**
  - Documents are split into smaller chunks, and each chunk is embedded separately.
  - This improves retrieval accuracy but increases storage and computational requirements.

##### Best Practices for Chunking:
- **Sentence-level chunks**: Too granular, losing context.
- **Paragraph-level chunks**: Works well if paragraphs are concise.
- **Overlapping chunks**: Improves context retention by including nearby text.
- **Adding titles or surrounding text**: Helps provide better contextual understanding.

As the field advances, more dynamic and LLM-based chunking methods are expected to emerge.


# Reranking

A reranker takes in the search query and a number of search results, and returns
the optimal ordering of these documents so the most relevant ones to the query are
higher in ranking. Cohere’s Rerank endpoint is a simple way to start using a first
reranker. We simply pass it the query and texts and get the results back. We don’t
need to train or tune it:

In [ ]:
query = "how precise was the science"
results = co.rerank(query=query, documents=texts, top_n=3, return_documents=True)
results.results

In [ ]:
for idx, result in enumerate(results.results):
    print(idx, result.relevance_score , result.document.text)

0 0.1698185 It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics

1 0.07004896 The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014

2 0.0043994132 Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar

### Difference Between Dense Retrieval and Reranking

| Feature         | **Dense Retrieval** | **Reranking** |
|---------------|-----------------|----------------|
| **Purpose** | Retrieves the most relevant documents from a large corpus. | Reorders the retrieved documents to improve ranking accuracy. |
| **Process** | Uses vector embeddings and similarity search to retrieve top-k candidates. | Takes the top-k retrieved documents and refines their ranking based on a more sophisticated scoring model. |
| **Speed** | Fast, optimized for large-scale retrieval. | Slower, as it applies a second-stage ranking process. |
| **Computational Cost** | Lower, as it relies on approximate nearest neighbor (ANN) search. | Higher, as it often uses transformer models or deep learning for precise ranking. |
| **Model Type** | Typically based on **bi-encoder** architectures (e.g., SBERT, DPR). | Often uses **cross-encoder** architectures (e.g., BERT-based rerankers). |
| **Strengths** | Efficient for large document collections; good for first-stage retrieval. | Improves precision by considering fine-grained contextual relationships between query and documents. |
| **Weaknesses** | May retrieve irrelevant results if the embedding similarity is misleading. | Computationally expensive and requires processing each query-document pair separately. |
| **Example Use Case** | Finding a set of potentially relevant articles for a query. | Refining the ranking of retrieved articles to show the most relevant ones at the top. |

#### How They Work Together
1. **Dense Retrieval (First Stage)**: Quickly retrieves the top-k most relevant documents using embeddings.
2. **Reranking (Second Stage)**: Uses a more detailed model to reorder these documents for better relevance.

This two-step approach balances **efficiency** (dense retrieval) and **accuracy** (reranking), making it a common pipeline in information retrieval systems. 🚀


# Retrieval-Augmented Generation

A basic RAG pipeline is made up of a search step followed by a grounded
generation step where the LLM is prompted with the question and the information
retrieved from the search step.

RAG systems incorporate search capabilities in addition to generation capabilities.
They can be seen as an improvement to generation systems because they reduce
their hallucinations and improve their factuality. They also enable use cases of “chat
with my data” that consumers and companies can use to ground an LLM on internal
company data, or a specific data source of interest (e.g., chatting with a book).
This also extends to search systems. More search engines are incorporating an LLM
to summarize results or answer questions submitted to the search engine. Examples
include Perplexity, Microsoft Bing AI, and Google Gemini.

##  Example: RAG with Local Models

In [1]:
!wget https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf


--2025-03-10 19:37:43--  https://huggingface.co/microsoft/Phi-3-mini-4k-instruct-gguf/resolve/main/Phi-3-mini-4k-instruct-q4.gguf
Resolving huggingface.co (huggingface.co)... 2600:9000:225f:ac00:17:b174:6d00:93a1, 2600:9000:225f:4600:17:b174:6d00:93a1, 2600:9000:225f:6800:17:b174:6d00:93a1, ...
Connecting to huggingface.co (huggingface.co)|2600:9000:225f:ac00:17:b174:6d00:93a1|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/41/c8/41c860f65b01de5dc4c68b00d84cead799d3e7c48e38ee749f4c6057776e2e9e/8a83c7fb9049a9b2e92266fa7ad04933bb53aa1e85136b7b30f1b8000ff2edef?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27Phi-3-mini-4k-instruct-q4.gguf%3B+filename%3D%22Phi-3-mini-4k-instruct-q4.gguf%22%3B&Expires=1741635464&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTc0MTYzNTQ2NH19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zLzQxL2M4LzQxYzg2MGY2NWIwMWRlNWRjNGM2OGIwMGQ

## Loading the generation model

Using llama.cpp, llama-cpp-python, and LangChain, we load the text generation
model:

In [3]:
from langchain import LlamaCpp
import multiprocessing

# Ensure your model file is CPU-compatible (q4, q5 quantized models recommended)
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-q4.gguf",  # Ensure you're using a CPU-friendly model
    n_gpu_layers=0,  # Force CPU execution (prevents Metal GPU issues)
    n_batch=256,  # Optimize batch size for CPU performance
    n_threads=multiprocessing.cpu_count(),  # Use all available CPU cores
    max_tokens=500,
    n_ctx=2048,  # Reduce context size if memory issues occur
    seed=42,
    verbose=False
)

# Check available CPU cores
print(f"Using {multiprocessing.cpu_count()} CPU cores")


llama_init_from_model: n_ctx_per_seq (2048) < n_ctx_train (4096) -- the full capacity of the model will not be utilized
ggml_metal_init: skipping kernel_get_rows_bf16                     (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_1row              (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_f32_l4                (not supported)
ggml_metal_init: skipping kernel_mul_mv_bf16_bf16                  (not supported)
ggml_metal_init: skipping kernel_mul_mv_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_mul_mm_bf16_f32                   (not supported)
ggml_metal_init: skipping kernel_mul_mm_id_bf16_f32                (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h64           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf16_h80           (not supported)
ggml_metal_init: skipping kernel_flash_attn_ext_bf

Using 8 CPU cores


## Loading the Embedding Model


In [5]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Embedding Model for converting text to numerical representations
embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-small-en-v1.5'
)

/var/folders/6j/6_13thh50qqgx2lhw_0qf67m0000gn/T/ipykernel_32391/1728599694.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(
/Users/aybikealkan/.pyenv/versions/3.10.6/envs/hands-on_llm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [6]:
embedding_model

HuggingFaceEmbeddings(client=SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': True}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': True, 'pooling_mode_mean_tokens': False, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False, 'pooling_mode_weightedmean_tokens': False, 'pooling_mode_lasttoken': False, 'include_prompt': True})
  (2): Normalize()
), model_name='BAAI/bge-small-en-v1.5', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

## Preparing the Vector Database


In [7]:

from langchain.vectorstores import FAISS

# Create a local vector database
db = FAISS.from_texts(texts, embedding_model)

In [7]:
texts

['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
 'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
 'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
 'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
 'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
 'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
 'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',
 'Interstellar uses extensive practical a

## The RAG prompt
A prompt template plays a vital part in the RAG pipeline. It is the central place where
we communicate the relevant documents to the LLM. To do so, we will create an
additional input variable named context that can provide the LLM with the retrieved
documents:

In [10]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA


# Create a prompt template
template = """<|user|>
Relevant information:
{context}

Provide a concise answer the following question using the relevant information provided above:
{question}<|end|>
<|assistant|>"""
prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# RAG Pipeline
rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(),
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=True
)

In [12]:
rag.invoke('Income generated')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'Income generated',
 'result': ' The information provided does not directly address the income generated from "Interstellar." However, we can infer that the collaboration of notable figures like Kip Thorne and companies such as Double Negative, along with its use of high-quality equipment like 35 mm movie film in Panavision anamorphic format and IMAX 70mm, contributed to the production\'s potential success. For specific financial details, additional information outside of this context would be required.'}

In [ ]:
rag.invoke('worldwide gross')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'worldwide gross',
 'result': ' The worldwide gross of Interstellar was over $677 million, making it the tenth-highest grossing film of 2014.'}

In [15]:
rag.invoke('is the movie scientifically trustable')



> Entering new RetrievalQA chain...

> Finished chain.


{'query': 'is the movie scientifically trustable',
 'result': ' Yes, Interstellar is considered to be scientifically trustable as it has received praise from many astronomers for its accurate portrayal of theoretical astrophysics and is regarded by sci-fi experts as one of the best science fiction films of all time.'}

In [13]:
print(db.docstore._dict)

{'f5712b18-5792-47a2-bd47-43128b979f42': Document(id='f5712b18-5792-47a2-bd47-43128b979f42', metadata={}, page_content='Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan'), '1325b284-31f4-4926-95be-29365b60555a': Document(id='1325b284-31f4-4926-95be-29365b60555a', metadata={}, page_content='It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine'), 'f258f984-2a91-4389-8b2e-020689cfa508': Document(id='f258f984-2a91-4389-8b2e-020689cfa508', metadata={}, page_content='Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind'), '41b402a0-b8c4-4480-9e00-53214e599124': Document(id='41b402a0-b8c4-4480-9e00-53214e599124', metadata={}, page_content='Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan 

In [14]:
import numpy as np

# Get the FAISS index object
faiss_index = db.index

# Get all stored vectors
vectors = np.array([faiss_index.reconstruct(i) for i in range(faiss_index.ntotal)])

print("Shape of stored vectors:", vectors.shape)  # (num_texts, embedding_dim)
print("Sample vector:", vectors[0])  # Print the first stored vector


Shape of stored vectors: (15, 384)
Sample vector: [ 9.00124945e-03 -1.84804648e-02 -7.19212964e-02 -3.18553322e-03
  1.87151860e-02 -3.19610424e-02 -3.68364947e-03 -1.89144518e-02
  2.25169566e-02 -3.31748873e-02  8.86677578e-02 -5.94718046e-02
 -1.13017736e-02  3.32215764e-02 -4.97691743e-02  8.32819007e-03
  8.12381413e-03 -2.57822964e-02 -4.16799448e-02  4.70038392e-02
 -2.46361103e-02  1.39289545e-02  4.51603308e-02 -5.17833792e-02
 -1.02366330e-02  7.20374510e-02 -3.23839635e-02 -2.94736810e-02
 -8.58287737e-02 -1.28986612e-01  5.47142364e-02  3.67259770e-03
  8.90053716e-03  1.19207455e-02  1.51587185e-02 -6.56802952e-02
 -2.69270875e-02 -1.49785150e-02 -2.40695383e-02 -5.39990049e-03
 -6.64924039e-03 -2.08047908e-02  1.00863911e-02 -1.98865198e-02
 -6.92458823e-03  1.48276323e-02 -2.29625590e-02 -1.87566224e-02
  4.01440449e-02 -4.84026372e-02  5.38560152e-02 -1.17137298e-01
 -2.16658972e-02 -7.19817029e-03 -9.19066146e-02  7.47764334e-02
 -1.60866268e-02  4.84245643e-02  4.2750